**Task 1**

I renamed excel file and column's names due to some utf-8 problems while reading

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns

from pathlib import Path

from statsmodels.stats.proportion import proportions_ztest

In [3]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    '''Computation of p-value for z-test'''
    
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - stats.norm.cdf(z_stat)
    
def proportions_diff_z_stat_ind(n1=None, n2=None, cnt_pos1=None, cnt_pos2=None,
                                p1=None, p2=None, sample1=None, sample2=None):
    '''Compitation of z-stat for fractions of independent samples'''
    
    if sample1 is not None:
        n1 = len(sample1)
        cnt_pos1 = sum(sample1)
        #подсчет вероятноти для 1-й популяции
        p1 = cnt_pos1 / n1
    elif n1 is not None and cnt_pos1 is not None:
        p1 = cnt_pos1 / n1
    elif n1 is not None:
        raise ValueError('You give no sample, no cnt_pos1, so it must be p1 and n1')
        
    if sample2 is not None:
        n2 = len(sample2)
        cnt_pos2 = sum(sample2)
        #подсчет вероятноти для 2-й популяции
        p2 = cnt_pos2 / n2
    elif n2 is not None and cnt_pos2 is not None:
        p2 = cnt_pos2 / n2
    elif n2 is not None:
        raise ValueError('You give no sample, no cnt_pos1, so it must be p2 and n2')
        
    P = (p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [2]:
df = pd.read_csv(Path('data', 'statistics_2018.csv'))
df.head()

,count,speed,type
0,3,3.8,A
1,3,3.8,B
2,3,4.8,B
3,4,4.4,B
4,6,5.0,B


**1**

1)	Более 3 бракованных листов на партию выходит значимо чаще для стали марки А, чем для стали марки B.

In [11]:
df.loc[df['type']=='A', 'count'].mean(), df.loc[df['type']=='B', 'count'].mean()

(3.179856115107914, 3.36734693877551)

Так как речь идет о сравнении не самих средних, а больше или не больше, чем k, то перейду к бинарным векторам.
Ввиду того, что используются разные марки стали, то, по моему мнению, логично предположить, что выборки являются независимыми

$H_0: p_1 = p_2$
<br>
$H_1: p_1 > p_2$

Пусть $\alpha$=0.05

In [62]:
sample1 = np.where(df.loc[df['type']=='A', 'count'] > 3, 1, 0)
sample2 = np.where(df.loc[df['type']=='B', 'count'] > 3, 1, 0)

In [51]:
len(sample1), len(sample2), np.mean(sample1), np.mean(sample2)

(139, 98, 0.381294964028777, 0.4489795918367347)

In [67]:
z_stat = proportions_diff_z_stat_ind(sample1=sample1, sample2=sample2)
print(f"p-value: {proportions_diff_z_test(z_stat, 'greater')}")

p-value: 0.8516651432021004


In [27]:
#сверка с библиотечной реализацией
proportions_ztest(np.array([sum(sample1), sum(sample2)]), np.array([len(sample1), len(sample2)])
                  , alternative='larger')[1]

0.8516651432021004

Гипотеза $H_0$ не отвергается

**2**

2)	При скоростях прокатки более 4 м/с свыше 3 бракованных листов стали на партию выходит значимо чаще, чем при меньших скоростях прокатки. 

Так как в задании говорится про сталь марки "А", то рассмотрю ее.

Аналогично предыдущему заданию перейду к бинарным векторам. C одной стороны кажется, что раз используется одна и та же марка стали "А", то выборки можно считать зависимыми, но проблема в том, что при таких ограничениях объемы выборок различны, то есть эксперименты неравнозначны, в связи с этим буду рассматривать их как независимые, как и в №1

$H_0: p_1 = p_2$
<br>
$H_1: p_1 > p_2$

Пусть $\alpha$=0.05

In [56]:
sample1 = np.where(df.loc[(df['speed'] > 4) & (df['type']=='A'), 'count'] > 3, 1, 0)
sample2 = np.where(df.loc[(df['speed'] <= 4) & (df['type']=='A'), 'count'] > 3, 1, 0)

In [49]:
len(sample1), len(sample2), np.mean(sample1), np.mean(sample2)

(100, 39, 0.39, 0.358974358974359)

In [53]:
z_stat = proportions_diff_z_stat_ind(sample1=sample1, sample2=sample2)
print(f"p-value: {proportions_diff_z_test(z_stat, 'greater')}")

p-value: 0.8516651432021004


Гипотеза $H_0$ не отвергается

## Вывод:

Возможно, что я не совсем верно интерпретировал формулировку, но в такой интерпретации кажутся две вещи: необходим переход к бинарным векторам и сравнение против односторонней альтернативы. В таком случае получается, что дважды нулевая гипотеза не отверглась, несмотря на то, что в задании говорится, что надо обосновать обратное.
Также, может быть, что станки на которые проходили обработки партии стали разные, условно, один из станков может допускать меньшее количество брака даже при больших скоростях. Кажется, это еще один из аргументов использования именно тестов для именно независимых выборок